<a href="https://colab.research.google.com/github/wonkwonlee/a2w-kma-big-data-contest/blob/main/soil_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 라이브러리
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
# 경로 설정
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/')

# 토양도 전처리
토양도 데이터 전처리 과정을 위한 경상북도 47_1, 47_2와 경상남도 48의 3가지 csv 파일을 전처리한다.

먼저 토양도 파일의 양이 많고 서로 다른 지역이므로 먼저 경상북도 47_1부터 전처리를 시작한다. 

전처리 과정은 3개의 파일 모두 동일하다.

In [ ]:
# 파일 불러오기
soil_47_1 = pd.read_csv('data/soil_47_1_gsbd.csv', encoding = 'utf-8')
soil_47_2 = pd.read_csv('data/soil_47_2_gsbd.csv', encoding = 'utf-8')
soil_48 = pd.read_csv('data/soil_48_gsnd.csv', encoding = 'utf-8')

In [ ]:
soil_47_1

,OBJ_ID,ARA_XCRD,ARA_YCRD,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,LDMARK_STN,MAP_LABEL
0,1,280272.57000,266104.66000,1,11,799.2,21.8,164.8,2,12.0,3.0,2.0,20.0,2.0,2,L102,B₂
1,2,283750.14000,265050.01000,1,11,534.8,15.7,301.6,2,12.0,5.0,2.0,20.0,2.0,2,L102,B₂
2,3,281027.00000,265427.00000,1,11,663.0,21.7,156.6,2,12.0,5.0,2.0,20.0,2.0,2,L102,B₂
3,4,279833.45000,267538.73000,1,11,850.3,9.8,129.4,2,12.0,3.0,2.0,20.0,2.0,2,L102,B₂
4,5,280102.57000,266674.66000,1,11,832.9,18.9,55.8,2,12.0,3.0,2.0,20.0,2.0,2,L102,B₂
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221279,221280,346202.03125,396564.93750,3,31,597.0,35.6,288.8,1,3.0,2.0,2.0,2.0,2.0,2,L102,B₂
221280,221281,349057.71875,396495.78125,3,31,905.9,33.1,312.2,1,3.0,2.0,2.0,2.0,2.0,2,L102,B₂
221281,221282,346512.03125,396794.93750,3,31,728.3,29.4,303.0,1,3.0,2.0,2.0,2.0,2.0,2,L102,B₂
221282,221283,346507.03125,397274.93750,3,31,768.2,33.7,287.9,1,3.0,2.0,3.0,2.0,2.0,2,L102,B₂


In [ ]:
soil_47_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221284 entries, 0 to 221283
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   OBJ_ID      221284 non-null  int64  
 1   ARA_XCRD    221284 non-null  float64
 2   ARA_YCRD    221284 non-null  float64
 3   PRRCK_LARG  221284 non-null  int64  
 4   PRRCK_MDDL  221284 non-null  int64  
 5   LOCTN_ALTT  221284 non-null  float64
 6   LOCTN_GRDN  221284 non-null  float64
 7   EIGHT_AGL   221284 non-null  float64
 8   CLZN_CD     221284 non-null  int64  
 9   TPGRP_TPCD  150588 non-null  float64
 10  PRDN_FOM_C  150588 non-null  float64
 11  SLANT_TYP   150588 non-null  float64
 12  SLDPT_TPCD  147824 non-null  float64
 13  SCSTX_CD    147824 non-null  float64
 14  SLTP_CD     221284 non-null  int64  
 15  LDMARK_STN  221284 non-null  object 
 16  MAP_LABEL   221284 non-null  object 
dtypes: float64(10), int64(5), object(2)
memory usage: 28.7+ MB


전체 자료의 크기는 221284 rows × 17 columns 이므로 221284개의 항목과 17개의 변수로 구성됨을 알 수 있다.

OBJ_ID의 경우 인덱스의 역할이므로 지우도록 한다.

![변수](https://user-images.githubusercontent.com/28593767/117817943-12d08180-b2a3-11eb-8169-d2f303afe79c.png)


In [ ]:
soil_47_1 = soil_47_1.drop(['OBJ_ID'], axis=1)
soil_47_1

,ARA_XCRD,ARA_YCRD,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,LDMARK_STN,MAP_LABEL
0,280272.57000,266104.66000,1,11,799.2,21.8,164.8,2,12.0,3.0,2.0,20.0,2.0,2,L102,B₂
1,283750.14000,265050.01000,1,11,534.8,15.7,301.6,2,12.0,5.0,2.0,20.0,2.0,2,L102,B₂
2,281027.00000,265427.00000,1,11,663.0,21.7,156.6,2,12.0,5.0,2.0,20.0,2.0,2,L102,B₂
3,279833.45000,267538.73000,1,11,850.3,9.8,129.4,2,12.0,3.0,2.0,20.0,2.0,2,L102,B₂
4,280102.57000,266674.66000,1,11,832.9,18.9,55.8,2,12.0,3.0,2.0,20.0,2.0,2,L102,B₂
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221279,346202.03125,396564.93750,3,31,597.0,35.6,288.8,1,3.0,2.0,2.0,2.0,2.0,2,L102,B₂
221280,349057.71875,396495.78125,3,31,905.9,33.1,312.2,1,3.0,2.0,2.0,2.0,2.0,2,L102,B₂
221281,346512.03125,396794.93750,3,31,728.3,29.4,303.0,1,3.0,2.0,2.0,2.0,2.0,2,L102,B₂
221282,346507.03125,397274.93750,3,31,768.2,33.7,287.9,1,3.0,2.0,3.0,2.0,2.0,2,L102,B₂


결측치는 TPGRP_TPCD, PRDN_FOM_C,	SLANT_TYP,	SLDPT_TPCD,	SCSTX_CD의 5개 열에서 발견되었다.

결측 데이터 5개 열 이해
* 지형구분코드 TPGRP_TPCD
  + 산정, 산능성, 산복 등 (01 ~ 12, 99 기타)
* 사면형태코드 PRDN_FOM_CD  
  + 1
* 경사형코드 SLANT_TYP_CD
  + 1 상승사면, 2 평행사면, 3 하강사면
* 토심구분코드 SLDPT_TPCD
  + 10 천, 20 경, 30 심
* 토성코드 SCSTX_CD
  + 사양토, 양토, 미사질양토 등 (01 ~ 11, 99 기타)

In [ ]:
soil_47_1.isnull().sum()

ARA_XCRD          0
ARA_YCRD          0
PRRCK_LARG        0
PRRCK_MDDL        0
LOCTN_ALTT        0
LOCTN_GRDN        0
EIGHT_AGL         0
CLZN_CD           0
TPGRP_TPCD    70696
PRDN_FOM_C    70696
SLANT_TYP     70696
SLDPT_TPCD    73460
SCSTX_CD      73460
SLTP_CD           0
LDMARK_STN        0
MAP_LABEL         0
dtype: int64